# Elementary feature interpretation

* Linear classifier learns a weight for every feature
* This weight (to a degree) correlates with the importance of this feature
* Study the attributes of a trained `LinearSVC` here https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
* `coef_` seems to be the right choice    

In [1]:
#Load the previously saved model
import pickle
import sklearn
with open("saved_model.pickle","rb") as f:
    classifier,vectorizer=pickle.load(f)
    
print("classifier weights:",classifier.coef_)
print("their shape",classifier.coef_.shape)

classifier weights: [[-0.00147616 -0.01517204 -0.00042053 ...  0.00023795 -0.0008693
  -0.00070104]]
their shape (1, 68322)


* We will need to know which weight corresponds to which feature
* The vectorizer has this information
* Study the attributes here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
* `vocabulary` seems to be the right choice

In [2]:
print(vectorizer.vocabulary_)

{'love': 36060, 'it': 31685, 'when': 66432, 'they': 60687, 'actually': 1489, 'do': 17659, 'sports': 56993, 'story': 57895, 'well': 66262, 'so': 56032, 'many': 37210, 'in': 30116, 'the': 60565, 'past': 44310, 'have': 27488, 'been': 5944, 'hokey': 28537, 'was': 65926, 'embarrassing': 19633, 'to': 61273, 'watch': 65973, 'not': 41942, 'this': 60764, 'one': 42745, 'just': 32658, 'genuinely': 24849, 'nice': 41478, 'movie': 40227, 'an': 2861, 'old': 42648, 'fashioned': 21844, 'type': 62824, 'of': 42515, 'and': 2944, 'based': 5550, 'on': 42730, 'real': 49103, 'life': 35256, 'guy': 26688, 'did': 16689, 'exactly': 20886, 'what': 66392, 'dennis': 16001, 'quaid': 48202, 'film': 22423, 'he': 27574, 'plays': 45876, 'high': 28195, 'school': 52960, 'coach': 11859, 'who': 66567, 'is': 31599, 'talked': 59697, 'into': 31288, 'trying': 62472, 'out': 43215, 'late': 34490, 'athletically': 4341, 'speaking': 56639, 'become': 5886, 'pitcher': 45679, 'professional': 47325, 'baseball': 5548, 'eventually': 20769,

* The vocabulary is a dictionary: feature -> index
* We will need it the other way, i.e. we will need to ask using an index, and get the feature

In [3]:
#Reverse the dictionary
index2feature={}
for feature,idx in vectorizer.vocabulary_.items():
    assert idx not in index2feature #This really should hold
    index2feature[idx]=feature
#Now we can query index2feature to get the feature names as we need

* We need now to sort the classifier weights
* ...and keep the information about which features (indices) they correspond to
* So a simple `sort()` does not cut it, we would not keep the indices

In [4]:
# Solution 1:

# make a list of (weight, index), sort it
lst=[]
for idx,weight in enumerate(classifier.coef_[0]):
    lst.append((weight,idx))
lst.sort() #sort

#Print first few and last few
for weight,idx in lst[:30]: #first 30 (ie lowest weight)
    print(index2feature[idx])
print("----------------------------------------------------")
#Take the last 30 (lst[-30:]) but these now come from weakest to strongest
#so reverse the list using [::-1]
for weight,idx in lst[-30:][::-1]:
    print(index2feature[idx])


worst
bad
awful
waste
boring
poor
nothing
terrible
worse
dull
stupid
poorly
unfortunately
supposed
no
horrible
script
disappointing
instead
ridiculous
annoying
mess
fails
avoid
minutes
disappointment
oh
lame
badly
plot
----------------------------------------------------
great
excellent
best
perfect
wonderful
amazing
fun
loved
love
enjoyed
favorite
beautiful
today
brilliant
superb
definitely
job
well
highly
also
both
enjoy
still
fantastic
bit
very
liked
makes
enjoyable
especially


In [5]:
# Solution #2
# Numpy can help us
# argsort gives a sequence of indices that sort an array
import numpy

indices=numpy.argsort(classifier.coef_[0])
print(indices)
for idx in indices[:30]:
    print(index2feature[idx])
print("-------------------------------")
for idx in indices[::-1][:30]: #you can also do it the other way round, reverse, then pick
    print(index2feature[idx])

[67290  4996  4781 ...  6393 20938 26060]
worst
bad
awful
waste
boring
poor
nothing
terrible
worse
dull
stupid
poorly
unfortunately
supposed
no
horrible
script
disappointing
instead
ridiculous
annoying
mess
fails
avoid
minutes
disappointment
oh
lame
badly
plot
-------------------------------
great
excellent
best
perfect
wonderful
amazing
fun
loved
love
enjoyed
favorite
beautiful
today
brilliant
superb
definitely
job
well
highly
also
both
enjoy
still
fantastic
bit
very
liked
makes
enjoyable
especially


* This seems to work like charm!
* We can sample the features across the range to get some further idea
* let's take every 100th feature

In [6]:
indices=numpy.argsort(classifier.coef_[0])

for idx in indices[::100]:
    print(index2feature[idx],end=", ")

worst, okay, ugly, want, killed, baldwin, category, cult, shouting, line, star, seagal, miles, boll, toilet, blows, toys, incestuous, objectively, ripping, yay, dafoe, misses, sally, ripe, resort, sexist, dies, closeups, cape, jumbo, flashback, genocide, reruns, goodfellas, noel, beaton, mishmash, annette, feminism, snore, henson, gravitas, stalked, cavorting, tortured, joaquin, cheerleader, fondling, missteps, tarr, physique, yanks, divorced, perched, 2d, handling, affords, dirt, brides, mobster, putrid, fleischer, flower, babe, dips, clocked, bah, zuckerman, angled, assailant, radiating, lieu, ren, hairstyle, thomson, sabotaging, tomas, martyr, belligerent, seller, fests, cooking, kamar, stratten, ungratefully, extraterrestrial, abiding, bonjour, unadulterated, scarred, jōb, submarine, ergo, naturalism, emblazered, stylised, clifton, muscle, ellissen, weights, exterminated, taryn, ye, negras, wimps, gabel, modernized, smoky, weezer, playwright, moderately, liz, famously, prima, bauch

* What have we learned?
* Most of the features seem to form a mass without a strong correlation with the sentiment
* Only the very extremes of the list seem to be strongly sentiment-biased